In [1]:
!python3 -m pip install beautifulsoup4 tqdm lxml

# alternative to selenium
# !python3 -m pip install playwright

In [22]:
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import requests
import urllib.request
import pandas as pd
import time

SEASONS = list(range(2012,2022))

def scrape_season(season):
    if season == 2022:
        url = "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"
    else:
        url = f"https://fbref.com/en/comps/9/{season}-{season+1}/schedule/{season}-{season+1}-Premier-League-Scores-and-Fixtures"
    hdr = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
    req = urllib.request.Request(url, headers=hdr)
    html = urllib.request.urlopen(req)

    # access relevant parts of the page via beautiful soup
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find('table')
    header_row = table.find('thead')
    body = table.find('tbody')

    # use getText()to extract the headers into a list
    headers = [*[th.getText() for th in header_row.findAll('th')], 'Season']

    # iterate through table row by row and scrape player data
    data = [[i, *[td.getText() for td in row.findAll('td')], int(season)] for i, row in enumerate(body.findAll('tr'))]

    # NOTE: sports reference does not allow more than 20 requests per minute
    # so we must sleep for 5 seconds
    time.sleep(5)
    df = pd.DataFrame(data, columns=headers)
    df = df[[col for col in df.columns if col != 'xG']]
    return df

# iteratively scrape each season worth of player stats and concatenate data into 1 dataframe
df = pd.DataFrame()
for season in tqdm(SEASONS):
    df = pd.concat([df, scrape_season(season)], ignore_index=True, axis=0)

df = df[df['Attendance'] != '']

100%|██████████| 10/10 [00:55<00:00,  5.52s/it]


In [26]:
df = df.reset_index(drop=True)
df.to_csv('epl-attendance.csv')